# Descriptors

In [1]:
# non data vs data descriptors
# writing custom descriptors
# avoiding common language pitfalls
# weak references and weak dictionaries

In [2]:
from datetime import datetime

In [3]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        return datetime.utcnow().isoformat()

In [4]:
class Logger:
    current_time = TimeUTC()

In [5]:
Logger.__dict__

mappingproxy({'__module__': '__main__',
              'current_time': <__main__.TimeUTC at 0x112b01d90>,
              '__dict__': <attribute '__dict__' of 'Logger' objects>,
              '__weakref__': <attribute '__weakref__' of 'Logger' objects>,
              '__doc__': None})

In [6]:
Logger.current_time

'2024-10-19T00:45:52.891188'

In [7]:
l = Logger()
l.current_time

'2024-10-19T00:45:52.895659'

In [8]:
from random import choice, seed

In [9]:
class Deck:
    @property
    def suit(self):
        return choice(('Spade', 'Heart', 'Diamond', 'Club'))
    
    @property
    def card(self):
        return choice(tuple('234567889JQKA')+('10',))

In [10]:
d = Deck()

In [11]:
seed(0)

for _ in range(10):
    print(d.card, d.suit)

10 Club
A Club
2 Diamond
9 Club
8 Diamond
8 Diamond
J Heart
9 Heart
6 Heart
A Spade


In [12]:
class Choice:
    def __init__(self, *choices):
        self.choices = choices
        
    def __get__(self, instance, owner_class):
        return choice(self.choices)
    
    

In [13]:
class Deck:
    suit = Choice('Spade', 'Heart', 'Diamond', 'Club')
    card = Choice(*'23456789JQKA', '10')

In [14]:
seed(0)
d = Deck()
for _ in range(10):
    print(d.card, d.suit)

8 Club
2 Diamond
J Club
8 Diamond
9 Diamond
Q Heart
J Heart
6 Heart
10 Spade
Q Diamond


In [15]:
class Dice:
    die_1 = Choice(1,2,3,4,5,6)
    die_2 = Choice(1,2,3,4,5,6)
    die_3 = Choice(1,2,3,4,5,6)

In [16]:
seed(0)

dice = Dice()

for _ in range(10):
    print(dice.die_1, dice.die_2, dice.die_3)

4 4 1
3 5 4
4 3 4
3 5 2
5 2 3
2 1 5
3 5 6
5 2 3
1 6 1
6 3 4


## Getters and setters 

In [17]:
from datetime import datetime

In [18]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        print(f'__get__ called, self = {self}, instance = {instance}, owner_class={owner_class}')
        return datetime.utcnow().isoformat()

In [19]:
class Logger1:
    current_time = TimeUTC()
    
class Logger2:
    current_time = TimeUTC()

In [20]:
Logger1.current_time

__get__ called, self = <__main__.TimeUTC object at 0x112b1bd90>, instance = None, owner_class=<class '__main__.Logger1'>


'2024-10-19T00:45:52.951072'

In [21]:
getattr(Logger1, 'current_time')

__get__ called, self = <__main__.TimeUTC object at 0x112b1bd90>, instance = None, owner_class=<class '__main__.Logger1'>


'2024-10-19T00:45:52.955769'

In [22]:
Logger2.current_time

__get__ called, self = <__main__.TimeUTC object at 0x112b3c710>, instance = None, owner_class=<class '__main__.Logger2'>


'2024-10-19T00:45:52.960309'

In [23]:
l1 = Logger1()

In [24]:
print(hex(id(l1)))

0x112afa210


In [25]:
l1.current_time

__get__ called, self = <__main__.TimeUTC object at 0x112b1bd90>, instance = <__main__.Logger1 object at 0x112afa210>, owner_class=<class '__main__.Logger1'>


'2024-10-19T00:45:52.972664'

In [26]:
l2 = Logger1()

In [27]:
l2.current_time

__get__ called, self = <__main__.TimeUTC object at 0x112b1bd90>, instance = <__main__.Logger1 object at 0x112b3e3d0>, owner_class=<class '__main__.Logger1'>


'2024-10-19T00:45:52.986913'

In [28]:
print(hex(id(l2)))

0x112b3e3d0


In [29]:
l2 = Logger2()
print(hex(id(l2)))
l2.current_time

0x112b15d10
__get__ called, self = <__main__.TimeUTC object at 0x112b3c710>, instance = <__main__.Logger2 object at 0x112b15d10>, owner_class=<class '__main__.Logger2'>


'2024-10-19T00:45:52.994635'

In [30]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return datetime.utcnow().isoformat()

In [31]:
class Logger:
    current_time = TimeUTC()

In [32]:
Logger.current_time

In [33]:
l = Logger()

In [34]:
l.current_time

'2024-10-19T00:45:53.012888'

In [35]:
class Logger:
    @property
    def current_time(self):
        return datetime.utcnow().isoformat()

In [36]:
Logger.current_time

In [37]:
l = Logger()

In [38]:
l.current_time

'2024-10-19T00:45:53.026769'

In [39]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            print(f'__get__ called in {self}')
            return datetime.utcnow().isoformat()
        
class Logger:
    current_time = TimeUTC()

In [40]:
l1 = Logger()
l2 = Logger()

In [41]:
l1.current_time, l2.current_time

__get__ called in <__main__.TimeUTC object at 0x112b65510>
__get__ called in <__main__.TimeUTC object at 0x112b65510>


('2024-10-19T00:45:53.038901', '2024-10-19T00:45:53.038910')

In [42]:
class Countdown:
    def __init__(self, start):
        self.start = start+1
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        self.start-=1
        return self.start
        

In [43]:
class Rocket:
    countdown = Countdown(10)

In [44]:
# both will share same instance
rocket1 = Rocket()
rocket2 = Rocket()

In [45]:
rocket1.countdown

10

In [46]:
rocket2.countdown # 

9

In [47]:
rocket1.countdown

8

In [48]:
class IntegerValue:
    def __set__(self, instance, value):
        print(f'__set__ called, instance={instance}, value={value}')
        
    def __get__(self, instance, owner_class):
        if instance is None:
            print('__get__ called from calss')
        else:
            print(f'__get__ called, instance={instance}, owner = {owner_class}')

In [49]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [50]:
Point2D.x

__get__ called from calss


In [51]:
p = Point2D()

In [52]:
p.x

__get__ called, instance=<__main__.Point2D object at 0x112b6e4d0>, owner = <class '__main__.Point2D'>


In [53]:
p.x = 100

__set__ called, instance=<__main__.Point2D object at 0x112b6e4d0>, value=100


In [54]:
class IntegerValue:
    def __set__(self, instance, value):
        self._value = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self._value

In [55]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [56]:
p1 = Point2D()

In [57]:
p1.x = 1.1
p1.y = 2.2

In [58]:
p1.x, p1.y

(1.1, 2.2)

In [59]:
p2 = Point2D()

In [60]:
p2.x = 10.0

In [61]:
p1.x # even p1 value changes when p2 was updated

10.0

In [62]:
# This is a problem as value of attribute in one instance changes when value is updated in another instance.
# To resolve this both getter and setter should be aware of instances

## Using an instance properties

In [63]:
# assuming object is hashable
# create a dictionary
# __set__ & __get__ with instance key in dictionary

In [64]:
class IntegerValue:
    def __set__(self, instance, value):
        instance.stored_value = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return getattr(instance, 'stored_value', None)

In [65]:
class Point1D:
    x = IntegerValue()

In [66]:
p1, p2 = Point1D(), Point1D()

In [67]:
p1.x = 10.1
p2.x = 20.2

In [68]:
p1.x, p2.x

(10, 20)

In [69]:
p1.__dict__

{'stored_value': 10}

In [70]:
p2.__dict__

{'stored_value': 20}

In [71]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [72]:
p = Point2D()

In [73]:
p.x = 10.1

In [74]:
p.__dict__

{'stored_value': 10}

In [75]:
p.y = 20.2

In [76]:
p.y

20

In [77]:
p.__dict__

{'stored_value': 20}

In [78]:
p.x  # can't just hard code name for stored_name as it will update x & y value in same instance

20

In [79]:
class IntegerValue:
    def __init__(self, name):
        self.storage_name = '_' + name
        
    def __set__(self, instance, value):
        setattr(instance, self.storage_name, value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return getattr(instance, self.storage_name, None)

In [80]:
class Point2D:
    x = IntegerValue('x')
    y = IntegerValue('y')

In [81]:
p1, p2 = Point2D(), Point2D()

In [82]:
p1.x = 10.1
p1.y = 20.2

In [83]:
p1.__dict__

{'_x': 10.1, '_y': 20.2}

In [84]:
p2.__dict__

{}

In [85]:
p2.x = 100.1
p2.y = 200.2

In [86]:
p2.__dict__

{'_x': 100.1, '_y': 200.2}

In [87]:
p1 = Point2D()

In [88]:
p1._x = 100

In [89]:
p1.__dict__

{'_x': 100}

In [90]:
p1.x = 200

In [91]:
p1.__dict__ # _x got overwritten

{'_x': 200}

In [92]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[instance] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return self.values.get(instance, None)

In [93]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [94]:
p1 = Point2D()
p2 = Point2D()

In [95]:
p1.x = 10.1
p1.y = 20.2

In [96]:
p1.x, p1.y

(10, 20)

In [97]:
Point2D.x.values

{<__main__.Point2D at 0x112b7bdd0>: 10}

In [98]:
Point2D.y.values

{<__main__.Point2D at 0x112b7bdd0>: 20}

In [99]:
p2.x = 100.1
p2.y = 200.2

In [100]:
p2.x, p2.y

(100, 200)

In [101]:
Point2D.x.values

{<__main__.Point2D at 0x112b7bdd0>: 10, <__main__.Point2D at 0x112afbd90>: 100}

In [102]:
# there is a potential memry leak in this approach^^

## Strong and weak References 

In [1]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [2]:
# when reference count of strong reference goes to 0 then object is destroyed

In [3]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return f'Person(name = {self.name})'

In [4]:
p1 = Person('abc')

In [5]:
p2 = p1

In [6]:
p1 is p2, id(p1), id(p2)

(True, 4496863760, 4496863760)

In [7]:
ref_count(id(p1))

2

In [8]:
del p2

In [9]:
ref_count(id(p1))

1

In [10]:
p1_id = id(p1)

In [11]:
ref_count(p1_id)

1

In [12]:
del p1

In [13]:
ref_count(p1_id) 

2

In [14]:
# weak references - strong refernce count is 0  then object is deleted. In that case weak reference 
# count gets deleted as well


In [15]:
import weakref

In [16]:
p1 = Person('abc')

In [17]:
p1_id  = id(p1)

In [18]:
ref_count(p1_id)

1

In [19]:
p2 = p1

In [20]:
ref_count(p1_id)

2

In [21]:
weak1 = weakref.ref(p1)

In [22]:
ref_count(p1_id) # still 2 not changed after weak reference

2

In [23]:
weak1 

<weakref at 0x10c0a82c0; to 'Person' at 0x10c092d90>

In [24]:
hex(id(p1))

'0x10c092d90'

In [25]:
weak1 is p1

False

In [26]:
weak1() is p1

True

In [27]:
print(weak1())

Person(name = abc)


In [28]:
ref_count(p1_id)

2

In [32]:
weak1()

Person(name = abc)

In [33]:
ref_count(p1_id) 

9

In [34]:
# re-rrun kernal to reduce refernce because of weak1 
# don't run weak1()

In [29]:
ref_count(p1_id)

2

In [30]:
p3 = weak1()

In [31]:
ref_count(p1_id)

3

In [32]:
del p3

In [33]:
ref_count(p1_id)

2

In [34]:
del p2

In [35]:
ref_count(p1_id)

1

In [36]:
print(weak1())

Person(name = abc)


In [37]:
weak1

<weakref at 0x10c0a82c0; to 'Person' at 0x10c092d90>

In [38]:
del p1

In [39]:
ref_count(p1_id)

4473038752

In [40]:
weak1

<weakref at 0x10c0a82c0; dead>

In [41]:
result = weak1()

In [42]:
print(result)

None


In [44]:
l = [1,2,3]
try:
    w = weakref.ref(l)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'list' object


In [45]:
# most of the built-in types doesn't supprot weak references

In [46]:
p1 = Person('abc')

In [48]:
d = weakref.WeakKeyDictionary()

In [49]:
ref_count(id(p1))

1

In [50]:
n  = {p1:'abc'}

In [51]:
ref_count(id(p1))

2

In [52]:
del n

In [53]:
ref_count(id(p1))

1

In [54]:
d[p1] = 'abc'

In [55]:
ref_count(id(p1)) # now the object has one strong reference and one weak reference. 
# Count through this command is for one strong reference

1

In [56]:
weakref.getweakrefcount(p1)

1

In [57]:
d2 = weakref.WeakKeyDictionary()
d2[p1] = 'abc'

In [58]:
ref_count(id(p1)), weakref.getweakrefcount(p1)

(1, 2)

In [59]:
p1.__weakref__

<weakref at 0x10c0a7330; to 'Person' at 0x10c093390>

In [60]:
hex(id(p1))

'0x10c093390'

In [62]:
list(d.keyrefs())

[<weakref at 0x10c1967a0; to 'Person' at 0x10c093390>]

In [63]:
del p1 # delete on and only strong reference

In [64]:
list(d.keyrefs()) # weak references are deleted as well

[]

In [65]:
 d['python'] = 'test' # Can't create weak reference to string object

TypeError: cannot create weak reference to 'str' object

# Back to instance properties

In [66]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[instance] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)
    

In [67]:
import weakref

In [68]:
class IntegerValue:
    def __init__(self):
        self.values = weakref.WeakKeyDictionary()
        
    def __set__(self, instance, value):
        self.values[instance] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)
    

In [69]:
class Point:
    x = IntegerValue()

In [70]:
p = Point()

In [71]:
print(hex(id(p)))

0x10d021210


In [72]:
p.x = 100.1

In [73]:
p.x

100

In [76]:
Point.x.values.keyrefs()

[<weakref at 0x10bdbdad0; to 'Point' at 0x10d021210>]

In [77]:
del p 

In [78]:
Point.x.values.keyrefs()

[]

In [79]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(id(instance))
    

In [81]:
class Point:
    x = IntegerValue()
    
    def __init__(self, x):
        self.x = x
        
    def __eq__(self, other):
        return isinstance(other, point) and self.x == other.x

In [82]:
p = Point(10.1)

In [83]:
p.x

10.1

In [84]:
p.x = 20.2

In [85]:
p.x

20.2

In [86]:
id(p), Point.x.values

(4513306256, {4513306256: 20.2})

In [87]:
Point.x.values

{4513306256: 20.2}

In [88]:
del p 

In [89]:
Point.x.values # this is dead entry that sticks in dictionary

{4513306256: 20.2}

In [90]:
# need to remove the dead entry from dictionary as well

In [91]:
p = Point(10.1)

In [92]:
weak_p = weakref.ref(p)

In [93]:
ref_count(id(p))

1

In [94]:
del p 

In [95]:
weak_p

<weakref at 0x10cd6c2c0; dead>

In [96]:
def obj_destroyed(obj):
    print(f'{obj} has been destroyed')

In [98]:
p = Point(10.1)
w = weakref.ref(p, obj_destroyed)

In [99]:
del p 

<weakref at 0x10d042f70; dead> has been destroyed


In [104]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), int(value))
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.values[id(instance)]
            return self.value_tuple
        
    def _remove_object(self, weak_ref):
        print(f'removing dead entry for {weak_ref}')
    

In [105]:
class Point:
    x = IntegerValue()

In [106]:
p1 = Point()
p2 = Point()

In [107]:
p1.x, p2.x = 10.1, 100.1

In [108]:
ref_count(id(p1)), ref_count(id(p2))

(1, 1)

In [109]:
del p1

removing dead entry for <weakref at 0x10d030270; dead>


In [115]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), int(value))
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.values[id(instance)]
            return self.values[id(instance)][1]
        
    def _remove_object(self, weak_ref):
        reverse_lookup = [key for key, value in self.values.items()
                         if value[0] is weak_ref]
        
        if reverse_lookup:
            key = reverse_lookup[0]
            del self.values[key]
            
        for key, value in self.values.items():
            if value[0] is weak_ref:
                del self.values[key]
                break
    

In [116]:
class Point:
    x = IntegerValue()

In [117]:
p = Point()

In [118]:
p.x = 10.1

In [119]:
p.x

10

In [121]:
Point.x.values

{4513781840: (<weakref at 0x10d099e40; to 'Point' at 0x10d0ad850>, 10)}

In [122]:
del p

In [123]:
Point.x.values

{}

In [124]:
class Person:
    pass

In [125]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

In [126]:
hasattr(Person.__weakref__, '__get__')

True

In [127]:
p = Person()

In [129]:
hasattr(p,'__weakref__')

True

In [130]:
print(p.__weakref__)

None


In [131]:
w = weakref.ref(p)

In [132]:
print(p.__weakref__)

<weakref at 0x10d1023e0; to 'Person' at 0x10d0fbe50>


In [135]:
p1 = Person()

In [136]:
hasattr(p1, '__weakref__')

True

In [137]:
class Person:
    __slots__ = 'name'

In [138]:
Person.__dict__ # now __weakref & __dict__ are gone 

mappingproxy({'__module__': '__main__',
              '__slots__': 'name',
              'name': <member 'name' of 'Person' objects>,
              '__doc__': None})

In [139]:
p = Person()

In [140]:
hasattr(p, '__weakref__') # now we no longer have weakref attribute. So can't create weak reference to the object

False

In [141]:
w = weakref.ref(p) # we get exception

TypeError: cannot create weak reference to 'Person' object

In [142]:
# resovle to above issue
class Person:
    __slots__ = 'name', '__weakref__'

In [144]:
p = Person()

In [145]:
w = weakref.ref(p)

In [221]:
class ValidString:
    def __init__(self, min_length = 0, max_length = 255):
        self.data = {}
        self._min_length = min_length
        self._max_length = max_length
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError('Value must be a string')
        if len(value)< self._min_length:
            raise ValueError(f'Value should be atleast {self._min_length} chars')
        if len(value)> self._max_length:
            raise ValueError(f'Value cannot exceed {self._max_length} chars')
        self.data[id(instance)] = (weakref.ref(instance, self._finalize_instance), value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        
        else:
            value_tuple = self.data.get(id(instance))
            return value_tuple[1]
        
    def _finalize_instance(self, weak_ref):
        reverse_lookup = [key for key, value in self.data.items() if value[0] is weak_ref]
        if reverse_lookup:
            key = reverse_lookup[0]
            del self.data[key]

In [222]:
class Person:
    __slots__ = '__weakref__'
    
    firstname = ValidString(1, 100)
    lastname = ValidString(1, 100)
    
    
    def __eq__(self, other):
        return (
              isinstance(other, Person) and 
               self.firstname == other.firstname and 
               self.lastname ==other.lastname
        )

    
class BankAccount:
    __slots__ = '__weakref__'
    
    account_number = ValidString(5, 255)
    
    def __eq__(self, other):
        return (
              isinstance(other, BankAccount) and 
               self.account_number == other.account_number 
        )

In [223]:
p1 = Person()

In [224]:
try:
  p1.firstname = ''
except ValueError as ex:
    print(ex)

Value should be atleast 1 chars


In [225]:
p2 = Person()

In [226]:
p1.firstname,  p1.lastname = 'John', 'Parker'
p2.firstname,  p2.lastname = 'Robert', 'Jr'

In [227]:
b1, b2 = BankAccount(), BankAccount()

In [228]:
b1.account_number , b2.account_number = 'Savings', 'Checking'

In [229]:
p1.firstname, p1.lastname

('John', 'Parker')

In [230]:
p2.firstname, p2.lastname

('Robert', 'Jr')

In [231]:
b1.account_number, b2.account_number

('Savings', 'Checking')

In [232]:
Person.firstname.data

{4514561328: (<weakref at 0x10d214e50; to 'Person' at 0x10d16bd30>, 'John'),
 4514552352: (<weakref at 0x10d204ae0; to 'Person' at 0x10d169a20>, 'Robert')}

In [233]:
Person.lastname.data

{4514561328: (<weakref at 0x10d204d10; to 'Person' at 0x10d16bd30>, 'Parker'),
 4514552352: (<weakref at 0x10d204e00; to 'Person' at 0x10d169a20>, 'Jr')}

In [234]:
BankAccount.account_number.data

{4514558064: (<weakref at 0x10d2043b0; to 'BankAccount' at 0x10d16b070>,
  'Savings'),
 4514557344: (<weakref at 0x10d204810; to 'BankAccount' at 0x10d16ada0>,
  'Checking')}

In [235]:
del p1

In [236]:
del p2

In [237]:
del b1

In [238]:
del b2

In [239]:
Person.firstname.data

{}

In [240]:
ref_count(4514554704)

2

# The __set_name__ method


In [241]:
class ValidString:
    def __set_name__(self, owner_class, property_name):
        print(f'__set_name__: owner= {owner_class}, property_name = {property_name}')

In [242]:
class Person:
    name = ValidString() # called at the time of complication

__set_name__: owner= <class '__main__.Person'>, property_name = name


In [243]:
class ValidString:
    def __set_name__(self, owner_class, property_name):
        print(f'__set_name__: owner= {owner_class}, property_name = {property_name}')
        self.property_name = property_name
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        print(f'__get__ called for property {self.property_name} of instance {instance}')

In [244]:
class Person:
    first_name = ValidString()
    last_name = ValidString()

__set_name__: owner= <class '__main__.Person'>, property_name = first_name
__set_name__: owner= <class '__main__.Person'>, property_name = last_name


In [245]:
p = Person()

In [246]:
p.first_name 

__get__ called for property first_name of instance <__main__.Person object at 0x10d0e01d0>


In [251]:
class ValidString:
    
    def __init__(self, min_length):
        self.min_length = min_length
        
    def __set_name__(self, owner_class, property_name):
        print(f'__set_name__: owner= {owner_class}, property_name = {property_name}')
        self.property_name = property_name
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f'{self.property_name} must be a string')
        if self.min_length is not None and len(value) < self.min_length:
            raise ValueError(f'{self.property_name} must be atleast {self.min_length} chars')
        key = '_' + self.property_name
        setattr(instance, key, value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        key = '_'+ self.property_name
        return getattr(instance, key, None)
        print(f'__get__ called for property {self.property_name} of instance {instance}')

In [255]:
class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)

__set_name__: owner= <class '__main__.Person'>, property_name = first_name
__set_name__: owner= <class '__main__.Person'>, property_name = last_name


In [256]:
p = Person()

In [257]:
try:
    p.first_name = 'Alex'
    p.last_name = 'M'
except ValueError as ex:
    print(ex)

last_name must be atleast 2 chars


In [258]:
p.first_name = 'Alex'
p.last_name = 'Martell'

In [259]:
p.first_name, p.last_name

('Alex', 'Martell')

In [260]:
p.__dict__

{'_first_name': 'Alex', '_last_name': 'Martell'}

In [261]:
p = Person()
p.first_name = 'some data I need to store'

In [262]:
p.__dict__

{'_first_name': 'some data I need to store'}

In [263]:
p.first_name = 'Alex'

In [264]:
p.__dict__

{'_first_name': 'Alex'}

In [265]:
class BankAccount:
    apr = 10

In [267]:
b = BankAccount()

In [268]:
b.apr, b.__dict__

(10, {})

In [269]:
b.apr = 100

In [270]:
b.__dict__

{'apr': 100}

In [271]:
b.apr

100

In [300]:
class ValidString:
    
    def __init__(self, min_length):
        self.min_length = min_length
        
    def __set_name__(self, owner_class, property_name):
        print(f'__set_name__: owner= {owner_class}, property_name = {property_name}')
        self.property_name = property_name
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f'{self.property_name} must be a string')
        if self.min_length is not None and len(value) < self.min_length:
            raise ValueError(f'{self.property_name} must be atleast {self.min_length} chars')
        instance.__dict__[self.property_name] = value
        
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        print('__get__ called')
        return instance.__dict__.get(self.property_name, None)
      

In [301]:
class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)

__set_name__: owner= <class '__main__.Person'>, property_name = first_name
__set_name__: owner= <class '__main__.Person'>, property_name = last_name


In [302]:
p = Person()

In [303]:
p.__dict__

{}

In [304]:
p.first_name = 'Alex'

In [305]:
p.__dict__

{'first_name': 'Alex'}

In [306]:
p.first_name

__get__ called


'Alex'

## Property lookup resolutions

In [307]:
class IntegerValue:
    def __set__(self, instance, value):
        print('__set__ called..')
        
    def __get__(slef, instance, owner_class):
        print('__get__ called..')
        
        

In [308]:
class Point:
    x = IntegerValue()

In [309]:
p = Point()

In [310]:
p.x = 100

__set__ called..


In [311]:
p.x 

__get__ called..


In [312]:
p.__dict__

{}

In [313]:
p.__dict__['x'] = 'hello'

In [314]:
p.__dict__

{'x': 'hello'}

In [315]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        print('__get_called...')
        

In [316]:
class Logger:
    current_time = TimeUTC()

In [317]:
l = Logger()

In [318]:
l.current_time

__get_called...


In [319]:
l.__dict__

{}

In [320]:
l.__dict__['current_time']= 'hello'

In [321]:
l.__dict__

{'current_time': 'hello'}

In [322]:
l.current_time

'hello'

In [323]:
del l.__dict__['current_time']

In [324]:
l.__dict__

{}

In [325]:
l.current_time

__get_called...


In [326]:
class ValidString:
    def __init__(self, min_length = None):
        self.min_length = min_length
        
    def __set_name__(self, owner_class, property_name):
        self.property_name = property_name
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError(f'{self.property_name} must be string')
        if self.min_length is not None and len(value) < self.min_length:
            raise ValueError(f'{self.property_name} not long enough')
        instance.__dict__[self.property_name] = value
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        print(f'calling __get__ for {self.property_name}')
        return instance.__dict__.get(self.property_name, None)

In [327]:
class Person:
    first_name = ValidString(1)
    last_name = ValidString(2)

In [328]:
p = Person()

In [329]:
p.__dict__

{}

In [330]:
p.first_name = 'Alex'

In [331]:
p.first_name

calling __get__ for first_name


'Alex'

## Properties and descriptors

In [332]:
from numbers import Integral

In [333]:
class Person:
    @property
    def age(self):
        return getattar(self,'_age', None)
    
    @age.setter
    def age(self, value):
        if not isinstance(value, Integral):
            raise ValueError('age: must be an integer.')
        if value <0:
            raise ValueError('age: must be an non-negative integer.')
        self._age = value    

In [334]:
p = Person()


try:
    p.age = 10
except ValueError as ex:
    print(ex)

In [335]:
p.__dict__

{'_age': 10}

In [336]:
p.age = 10

In [337]:
p.__dict__

{'_age': 10}

In [346]:
class Person:
    
    def get_age(self):
        return getattr(self,'_age', None)
    
  
    def set_age(self, value):
        if not isinstance(value, Integral):
            raise ValueError('age: must be an integer.')
        if value <0:
            raise ValueError('age: must be an non-negative integer.')
        self._age = value 
        
    age = property(fget = get_age, fset = set_age)   

In [347]:
prop = Person.age

In [348]:
hasattr(prop, '__get__')

True

In [349]:
hasattr(prop, '__set__')

True

In [350]:
hasattr(prop, '__delete__')

True

In [351]:
p = Person()
p.age = 10
print(p.age)

10


In [352]:
class TimeUTC:
    @property
    def current_time(self):
        return 'current time'

In [353]:
t= TimeUTC()

In [354]:
hasattr(TimeUTC.current_time, '__get__')

True

In [355]:
hasattr(TimeUTC.current_time, '__set__')

True

In [356]:
t.current_time

'current time'

In [357]:
t.current_time = 'other' # because it doesn't have a set method to call but set is defined for all properties by default

AttributeError: property 'current_time' of 'TimeUTC' object has no setter

In [358]:
p = Person()

In [359]:
p.__dict__

{}

In [360]:
p.age = 10

In [361]:
p.__dict__

{'_age': 10}

In [363]:
p.__dict__['age'] = 100

In [364]:
p.__dict__

{'_age': 10, 'age': 100}

In [365]:
p.age 

10

In [366]:
class MakeProperty:
    def __init__(self, fget = None, fset= None):
        self.fget = fget
        self.fset = fset
        
    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
        
        
    def __get__(self, instance, owner_class):
        print('__get__ called..')
        if isinstance is None:
            return self
        if self.fget is None:
            raise AttributeError(f'{self.prop_name} is not readable')
        return self.fget(instance)
    
    
    def __set__(self, instance, value):
        print('__set__ called..')
        if self.fset is None:
            raise AttirbuteError(f'{self.prop_name} is not writable')
        self.fset(instance, value)
        
        

In [370]:
class Person:
    def get_name(self):
        print('get name called...')
        return getattr(self, '_name', None)
    
    def set_name(self, value):
        print('set name called...')
        self._name = value
        
    name = MakeProperty(fget = get_name, fset = set_name)    

In [371]:
p = Person()

In [372]:
p.name = 'Guido'

__set__ called..
set name called...


In [373]:
p.__dict__

{'_name': 'Guido'}

In [374]:
p.__dict__['name'] = 'Alex'

In [375]:
p.__dict__

{'_name': 'Guido', 'name': 'Alex'}

In [376]:
# using decorator approach


class Person:
    @MakeProperty
    def age(self):
        return 100


In [377]:
p = Person()

In [378]:
Person.age

__get__ called..


100

In [379]:
p.age

__get__ called..


100

In [380]:
class MakeProperty:
    def __init__(self, fget = None, fset= None):
        self.fget = fget
        self.fset = fset
        
    def __set_name__(self, owner_class, prop_name):
        self.prop_name = prop_name
        
        
    def __get__(self, instance, owner_class):
        print('__get__ called..')
        if isinstance is None:
            return self
        if self.fget is None:
            raise AttributeError(f'{self.prop_name} is not readable')
        return self.fget(instance)
    
    
    def __set__(self, instance, value):
        print('__set__ called..')
        if self.fset is None:
            raise AttirbuteError(f'{self.prop_name} is not writable')
        self.fset(instance, value)
        
    def setter(self, fset):
        self.fset = fset
        return self

In [381]:
class Person:
    @MakeProperty
    def age(self):
        return getattr(self, '_age', None)
    
    @age.setter
    def age(self, value):
        self._age = value

In [382]:
p = Person()

In [383]:
p.age = 100

__set__ called..


In [384]:
p.age

__get__ called..


100

In [385]:
p.__dict__

{'_age': 100}

## Function and descriptors 

In [386]:
def add(a,b):
    return a+b

In [387]:
hasattr(add, '__get__')

True

In [388]:
hasattr(add, '__set__')

False

In [390]:
import sys
me = sys.modules['__main__']

In [391]:
me

<module '__main__'>

In [392]:
f = add.__get__(None, me)

In [393]:
f

<function __main__.add(a, b)>

In [394]:
f is add

True

In [395]:
class Person:
    def __init__(self, name):
        self.name = name
      
    def say_hello(self):
        return f'{self.name} says hello!'

In [396]:
Person.say_hello

<function __main__.Person.say_hello(self)>

In [397]:
p = Person('Alex')

In [398]:
p.say_hello

<bound method Person.say_hello of <__main__.Person object at 0x10d46fbd0>>

In [399]:
bound_method = Person.say_hello.__get__(p, Person)

In [400]:
bound_method

<bound method Person.say_hello of <__main__.Person object at 0x10d46fbd0>>

In [401]:
f1 = p.say_hello
f2 = p.say_hello

In [402]:
f1, f2

(<bound method Person.say_hello of <__main__.Person object at 0x10d46fbd0>>,
 <bound method Person.say_hello of <__main__.Person object at 0x10d46fbd0>>)

In [403]:
f1 is f2

False

In [404]:
p.say_hello()

'Alex says hello!'

In [405]:
bound_method()

'Alex says hello!'

In [406]:
type(bound_method)

method

In [407]:
bound_method.__func__

<function __main__.Person.say_hello(self)>

In [409]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def say_hello(self):
        return f'{self.name} says hello'

In [410]:
def say_hello(self):
        return f'{self.name} says hello'

In [411]:
say_hello

<function __main__.say_hello(self)>

In [412]:
import types

In [413]:
help(types.MethodType)

Help on class method in module builtins:

class method(object)
 |  method(function, instance, /)
 |  
 |  Create a bound instance method object.
 |  
 |  Methods defined here:
 |  
 |  __call__(self, /, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __delattr__(self, name, /)
 |      Implement delattr(self, name).
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __hash__(self, /)
 |      Return hash(self).
 |  
 |  __le__(self, value, /)
 |      Return self<=value.
 |  
 |  __lt__(self, value, /)
 |      Return self<value.
 |  
 |  __ne__(self, value, /)
 |      Return self!=value.
 |  
 |  __reduce__(self, /)
 |      Helper for pickle.
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  __setattr__(self, name, value, /)
 |      Implement setattr(self

In [414]:
class Person:
    def __init__(self, name):
        self.name = name

In [415]:
p = Person('Alex')

In [417]:
m = types.MethodType(say_hello, p)

In [418]:
m

<bound method say_hello of <__main__.Person object at 0x10d3c71d0>>

In [419]:
p

In [420]:
m.__func__

<function __main__.say_hello(self)>

In [428]:
class MyFunc:
    def __init__(self, func):
        self._func = func
        
    def __get__(self, instance, owner_class):
        if instance is None:
            print('__get__ called from class')
            return self._func
        else:
            print('__get__ called form instance')
            return types.MethodType(self._func, instance)

In [429]:
def hello(self):
    print(f'{self.name} says hello')

In [430]:
class Person:
    def __init__(self, name):
        self.name = name
        
    say_hello = MyFunc(hello)    

In [431]:
Person.say_hello

__get__ called from class


<function __main__.hello(self)>

In [432]:
p = Person('Alex')

In [433]:
p.say_hello

__get__ called form instance


<bound method hello of <__main__.Person object at 0x10d495350>>